In [1]:
using thesis, PRMaps
using StatsPlots
using Pandas
using Healpix
using StatsPlots
using Plots
using LinearAlgebra
using Dates
using HDF5, JLD

import Stripeline as Sl

In [2]:
wmap = read_pickle("../instruments/wmap_instrument.pkl")
planck = read_pickle("../instruments/planck_instrument.pkl")
#quijote = read_pickle("../instruments/quijote_instrument.pkl")
swipe = read_pickle("../instruments/lspe_swipe_instrument.pkl")
strip = read_pickle("../instruments/lspe_strip_instrument.pkl")

strip = query(strip, :(frequency==43.0))
wmap = query(wmap, :(frequency==22.69))
planck = query(planck, :(frequency==28.4))

columns = ["instrument", "frequency", "depth_p", "depth_i"]

#instruments = concat([wmap, planck, quijote, swipe, strip])
instruments = concat([wmap, planck, strip, swipe])

instruments = instruments[columns]
instruments

,instrument,frequency,depth_p,depth_i
0,WMAP,22.69,294.697795,293.465609
0,lfi,28.40,210.000000,150.000000
0,LSPE/Strip,43.00,130.000000,183.847763
2,LSPE/SWIPE,145.00,10.000000,14.142136
3,LSPE/SWIPE,210.00,17.000000,24.041631
4,LSPE/SWIPE,240.00,34.000000,48.083261


In [3]:
nside = 64
sky_model = "c1s3d0"
t_start = DateTime(2022, 1, 1, 12, 0, 0)
days = 10

error = deg2rad(0.5/60)

cam_ang = Sl.CameraAngles()
 
setup = PRMaps.Setup(
    sampling_freq_Hz = 50.0,
    total_time_s = 24. * 3600. * days
    )
nothing

In [4]:
result = run_fgbuster(instruments, cam_ang, setup, sky_model, nside, t_start)
nothing

In [5]:
FINAL_RESULTS = Dict("ideal"=>result)

Dict{String, Dict{Any, Any}} with 1 entry:
  "ideal" => Dict("nit"=>13, "x"=>[1.58773, 18.0024, -2.98695, -0.0459959], "he…

In [6]:
labels = ["altitude_offset" "ground_offset" "fork" "zVAX&ω_0" "zVAX&ω_90" "zVAX&ω_180" "zVAX&ω_270"]

w2_TA = Sl.TelescopeAngles(wheel2ang_0_rad = error)
w3_TA = Sl.TelescopeAngles(wheel3ang_0_rad = error)
fork_TA = Sl.TelescopeAngles(forkang_rad = error)
wω_TA_0 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(0.0))
wω_TA_90 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(90.0))
wω_TA_180 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(180.0))
wω_TA_270 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(270.0))

tel_angs = [w2_TA, w3_TA, fork_TA, wω_TA_0, wω_TA_90, wω_TA_180, wω_TA_270]

nothing

In [7]:
for (i, tel_ang) in enumerate(tel_angs) 
    FINAL_RESULTS[labels[i]] = run_fgbuster_with_error(instruments, cam_ang, tel_ang, setup, sky_model, nside, t_start)
end

In [8]:
# Salvare il dizionario con tutti i risultati
JLD.save("fgbuster/results_lspe_planck_wmap_30arcsec.jld", FINAL_RESULTS)